# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

weather_data = pd.read_csv("output_data/cities.csv")
cities_pd = weather_data.dropna()


# Display sample data
cities_pd.head(20)

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,0,elk city,US,2023-03-28 12:03:24,35.4120,-99.4043,0.0,52.0,43.052,7.72
1,1,conceicao do araguaia,BR,2023-03-28 12:03:25,-8.2578,-49.2647,50.0,64.0,86.018,2.92
2,2,mataura,NZ,2023-03-28 12:03:25,-46.1927,168.8643,47.0,84.0,44.744,0.67
3,3,bredasdorp,ZA,2023-03-28 12:03:25,-34.5322,20.0403,38.0,67.0,65.714,6.37
4,4,dakar,SN,2023-03-28 12:03:25,14.6937,-17.4441,7.0,74.0,101.552,6.09
5,5,busselton,AU,2023-03-28 12:03:25,-33.6500,115.3333,0.0,65.0,61.250,6.25
6,6,guilin,CN,2023-03-28 12:03:25,25.2819,110.2864,100.0,90.0,56.264,1.65
7,7,nome,US,2023-03-28 12:03:25,64.5011,-165.4064,99.0,98.0,-0.526,2.36
8,8,bluff,NZ,2023-03-28 12:00:07,-46.6000,168.3333,97.0,63.0,45.068,2.90
9,9,qaanaaq,GL,2023-03-28 12:03:26,77.4840,-69.3632,79.0,70.0,-7.402,2.25


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# format the gmaps figure to a good size for display
# I played with these parameters quite a bit before settling on the chosen values
figure_layout ={
    'width': '1100px',
    'height': '400px',
    'margin': '0 auto 0 auto'
}

# assign formatted gmaps figure to a variable
fig = gmaps.figure(layout=figure_layout)

# create variable to hold values necessary for the heatmap layer in gmaps
locations = cities_pd[['Latitude','Longitude']]
rating = cities_pd['Humidity (%)'].astype(float)

# create the heat layer in gmaps
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                 dissipating=False, max_intensity=0,
                                 point_radius=5)

# add the heat layer to my fig variable
fig.add_layer(heat_layer)

#display figure with heatmap overlayed
fig

Figure(layout=FigureLayout(height='400px', margin='0 auto 0 auto', width='1100px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Filter vacation with zero cloudiness
vacation_no_cloud = cities_pd[cities_pd["Cloudiness (%)"] == 0]

# Filter vacation with max temp below 60 degrees F
vacation_below_60_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] < 60]

# Filter vacation with humidity above 50 %
vacation = vacation_below_60_degrees[vacation_below_60_degrees["Humidity (%)"] > 50]
# Set Index
perfect_vacation = vacation.reset_index()
del perfect_vacation["index"]
perfect_vacation

,City_ID,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,0,elk city,US,2023-03-28 12:03:24,35.4120,-99.4043,0.0,52.0,43.052,7.72
1,16,belyy yar,RU,2023-03-28 12:03:27,53.6039,91.3903,0.0,78.0,41.126,3.00
2,48,north platte,US,2023-03-28 12:03:32,41.1239,-100.7654,0.0,80.0,32.162,3.09
3,131,douglas,US,2023-03-28 12:00:46,41.3000,-96.1670,0.0,64.0,34.880,1.79
4,148,andenes,NO,2023-03-28 12:01:52,69.3143,16.1194,0.0,53.0,27.194,3.09
5,202,port hardy,CA,2023-03-28 12:03:51,50.6996,-127.4199,0.0,74.0,41.216,3.18
6,249,semey,KZ,2023-03-28 12:03:59,50.4111,80.2275,0.0,87.0,41.486,2.00
7,315,sisimiut,GL,2023-03-28 12:04:15,66.9395,-53.6735,0.0,66.0,14.216,2.35
8,426,dumas,US,2023-03-28 12:04:33,35.8656,-101.9732,0.0,51.0,41.882,2.06
9,524,grindavik,IS,2023-03-28 12:04:47,63.8424,-22.4338,0.0,51.0,36.338,8.23


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Hotel variable
hotels = []

params = {
    "apiKey":geoapify_key,
    "format":"json"
}
​
# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

# Print a message to follow up the airport search
print("Starting hotels search")
​
# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in perfect_vacation.iterrows():
​
    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    vacation = row["City"] + row["Country"]
​
    # Add the current city to the parameters
    params["text"] = f"{city}"
​
    # Make the API request
    response = requests.get(base_url, params=params)
    
    # Convert response to JSON
    response = response.json()
​
    # Extract latitude and longitude
    vacation.loc[index, "Lat"] = response["results"][0]["lat"]
    vacation.loc[index, "Lon"] = response["results"][0]["lon"]
    
    # Log the search results
    print(f"Coordinates for {city} fetched...")

SyntaxError: invalid non-printable character U+200B (301036250.py, line 8)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)